In [16]:
# Load vector database that was persisted earlier and check collection count in it
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
persist_directory = 'vector_db/chroma/'
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
print(vectordb._collection.count())

1267


In [20]:
question = "what is Quasarion?"
docs = vectordb.similarity_search(question,k=3)
len(docs)
for res in docs:
    print (res.page_content)
    print ('============')

Planet: Quasarion 
 
In the vast expanse of the Andromeda Galaxy, nestled amidst a cosmic ballet of stars, Quasarion emerge
s as a celestial jewel, a testament to the imagination's boundless creativity. This fictitious planet, with its e
nigmatic name echoing through the cosmic winds, beckons explorers and dreamers alike to embark on a j
ourney of discovery. Let's delve into the awe-inspiring wonders of Quasarion, a world that defies expectati
ons and captivates the soul. 
 
Physical Characteristics: 
 
Quasarion is a gas giant, its massive form dominating the space it occupies within the Ursa Cluster. With 
a diameter nearly ten times that of Jupiter, this colossal planet is a swirling masterpiece of atmospheric dy
namics. The surface of Quasarion is veiled in layers of vibrant gases, creating mesmerizing bands of colo
rs that shift and dance with the planet's eternal tempests. 
 
The planet boasts a unique ring system, a delicate network of iridescent particles that encircle Quasario

In [145]:
from transformers import AutoTokenizer, DistilBertForQuestionAnswering, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

# Load model directly

pipe = pipeline("question-answering", model=model, tokenizer=tokenizer,     temperature=0.2,
    max_new_tokens=512)

llm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs={"temperature": 0.2, "max_length": 1024},
)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [151]:
# from langchain.chains.question_answering import load_qa_chain
# chain = load_qa_chain(llm, chain_type="stuff",verbose=True)
import torch
query = "What is Quasarion ?"
matching_docs = vectordb.max_marginal_relevance_search(query)

context = ''
for i in matching_docs:
    context += " " + i.page_content

inputs = tokenizer(question, context[:512], return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()
if answer_start_index > answer_end_index:
    answer_end_index, answer_start_index = answer_start_index, answer_end_index
print (answer_start_index, answer_end_index)
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
print (predict_answer_tokens)
tokenizer.decode(predict_answer_tokens)

tensor(60) tensor(87)
tensor([ 9152, 21693, 12070,  2171, 17142,  2083,  1996, 14448,  7266,  1010,
        10272,  5644, 19264,  1998, 24726,  2015, 11455,  2000, 28866,  2006,
         1037,  1046,  2256,  5420,  1997,  5456,  1012,  2292])


'nigmatic name echoing through the cosmic winds, beckons explorers and dreamers alike to embark on a j ourney of discovery. let'